In [51]:
import numpy as np
import cmath
import math
from math import lgamma,exp
#from sklearn.preprocessing import normalize
from scipy.constants import epsilon_0 as ε_0, c, pi as π, e, hbar as ℏ
eV = e
from scipy.special import lpmn, lpmv, sph_jn, sph_yn, poch
from scipy.misc import factorial
from numpy import newaxis as ň
import sympy.physics.wigner as sw
debye = 3.33564e-30

I = np.identity(3)

In [3]:
from py_gmm.gmm import vec_trans as vc

In [14]:
vc.gaunt_xu(-15,55,-58,72,27)

(array([  2.64656585e+17,  -9.02060942e+17,   1.26120695e+18,
         -9.08154188e+17,   3.32877338e+17,  -3.99902372e+16,
         -9.37042460e+15,   1.93795427e+15,   4.15147444e+14,
         -4.50334569e+13,  -1.87310470e+13,  -6.89282471e+11,
          4.70652220e+11,   8.36238653e+10,   3.33171146e+08,
         -1.79374160e+09,  -2.62337431e+08,  -5.94969041e+06,
          3.03619342e+06,   4.52389705e+05,   2.13275693e+04,
         -1.46829323e+03,  -2.69213511e+02,  -1.39717873e+01,
          9.60086347e-03,   2.93425224e-02,   8.10789281e-04,
         -5.91092804e-06]), 0)

In [5]:
sw.gaunt(15,16,17,4,5,-9).evalf()

-0.0648286110541008

In [6]:
# NOT REALLY USING THIS NOW; JUST FOR BENCHMARKS
# from https://raw.githubusercontent.com/michael-hartmann/gaunt/master/gaunt.py

from __future__ import division
from math import lgamma,exp

def gaunt_a0(n,nu,m,mu):
    # eq. (20)
    return exp( lgamma(2*n+1)-lgamma(n+1)+lgamma(2*nu+1)-lgamma(1+nu)+lgamma(n+nu+1)-lgamma(2*n+2*nu+1) + lgamma(1+n+nu-m-mu)-lgamma(1+n-m)-lgamma(1+nu-mu) )


def gaunt_hartmann(n,nu,m,mu):
    """
    Determine Gaunt coefficients a(m, n, mu, nu, p) for m, n, mu and nu fixed.
    These coefficients can be used to express the product of two associated
    Legendre polynomials:

    P_n^m(x)*P_{nu}^{mu}(x) = a0 sum_{q=0}^{qmax} aq_tilde P_{n+nu-2q}^(m+mu)(x)
    
    Returns: qmax, a0, aq_tilde
    qmax is the upper bound of summation, a0 is the prefactor and aq_tilde is a
    list of normalized Gaunt coefficients.

    See [1] for more information, especially chapter 3. There is a brief
    outline how to calculate Gaunt coefficients at the end of the chapter.

    Ref.: [1] Y.-L. Xu, J. Comp. Appl. Math. 85, 53 (1997)
    """

    # eq. (28)
    Ap = lambda p:p*(p-1)*(m-mu)-(m+mu)*(n-nu)*(n+nu+1)

    # eq. (3)
    alpha = lambda p: ((p**2-(n+nu+1)**2)*(p**2-(n-nu)**2))/(4*p**2-1)

    # eq. (24)
    qmax = min(n,nu,(n+nu-abs(m+mu))//2)

    a0 = gaunt_a0(n,nu,m,mu)

    n4 = n+nu-m-mu
    a_tilde = [0]*(qmax+1)

    a_tilde[0] = 1
    if qmax == 0:
        return qmax,a0,a_tilde

    # eq. (29)
    a_tilde[1] = (n+nu-1.5)*(1-(2*n+2*nu-1)/(n4*(n4-1))*((m-n)*(m-n+1)/(2*n-1)+(mu-nu)*(mu-nu+1)/(2*nu-1)))
    if qmax == 1:
        return qmax,a0,a_tilde

    # eq. (35)
    a_tilde[2] = (2*n+2*nu-1)*(2*n+2*nu-7)/4*( (2*n+2*nu-3)/(n4*(n4-1)) * ( (2*n+2*nu-5)/(2*(n4-2)*(n4-3)) \
                * ( (m-n)*(m-n+1)*(m-n+2)*(m-n+3)/((2*n-1)*(2*n-3)) \
                + 2*(m-n)*(m-n+1)*(mu-nu)*(mu-nu+1)/((2*n-1)*(2*nu-1)) \
                + (mu-nu)*(mu-nu+1)*(mu-nu+2)*(mu-nu+3)/((2*nu-1)*(2*nu-3)) ) - (m-n)*(m-n+1)/(2*n-1) \
                - (mu-nu)*(mu-nu+1)/(2*nu-1) ) +0.5)


    for q in range(3,qmax+1):
        p = n+nu-2*q
        p1 = p-m-mu
        p2 = p+m+mu

        if Ap(p+4) != 0:
            # eqs. (26), (27)
            c0 = (p+2)*(p+3)*(p1+1)*(p1+2)*Ap(p+4)*alpha(p+1)
            c1 = Ap(p+2)*Ap(p+3)*Ap(p+4) \
               + (p+1)*(p+3)*(p1+2)*(p2+2)*Ap(p+4)*alpha(p+2) \
               + (p+2)*(p+4)*(p1+3)*(p2+3)*Ap(p+2)*alpha(p+3)
            c2 = -(p+2)*(p+3)*(p2+3)*(p2+4)*Ap(p+2)*alpha(p+4)
            a_tilde[q] = (c1*a_tilde[q-1] + c2*a_tilde[q-2])/c0
        else:
            if Ap(p+6) == 0:
                # eq. (30)
                a_tilde[q] = (p+1)*(p2+2)*alpha(p+2)*a_tilde[q-1] / ((p+2)*(p1+1)*alpha(p+1))
            else:
                # eq. (32), (33)
                c0 = (p+2)*(p+3)*(p+5)*(p1+1)*(p1+2)*(p1+4)*Ap(p+6)*alpha(p+1)
                c1 = (p+5)*(p1+4)*Ap(p+6)*(Ap(p+2)*Ap(p+3)+(p+1)*(p+3)*(p1+2)*(p2+2)*alpha(p+2))
                c2 = (p+2)*(p2+3)*Ap(p+2)*(Ap(p+5)*Ap(p+6)+(p+4)*(p+6)*(p1+5)*(p2+5)*alpha(p+5))
                c3 = -(p+2)*(p+4)*(p+5)*(p2+3)*(p2+5)*(p2+6)*Ap(p+2)*alpha(p+6)
                a_tilde[q] = (c1*a_tilde[q-1] + c2*a_tilde[q-2] + c3*a_tilde[q-3])/c0

    return qmax,a0,a_tilde

In [23]:
def q_max(m,n,μ,ν):
    return min(n,ν,(n+ν-abs(m+μ))/2)
    
# returns array with indices corresponding to q
# argument q does nothing for now
def a_q(m,n,μ,ν,q = None):
    qm=q_max(m,n,μ,ν)
    res, err= vc.gaunt_xu(m,n,μ,ν,qm)
    if(err):
        raise ValueError('Something bad in the fortran subroutine gaunt_xu happened')
    return res
    

In [66]:
def _sph_z_1(n,z):
    return sph_jn(n,z)[0]
def _sph_z_2(n,z):
    return sph_yn(n,z)[0]
def _sph_z_3(n,z):
    return (sph_jn(n,z)[0] + 1j*sph_yn(n,z)[0])
def _sph_z_4(n,z):
    return (sph_jn(n,z)[0] - 1j*sph_yn(n,z)[0])
_sph_z = [_sph_z_1,_sph_z_2,_sph_z_3,_sph_z_4]

# computes bessel/hankel functions for orders from 0 up to n; drops
# the derivatives which are also included in scipy.special.sph_jn/yn
def sph_zn(n, z, J=1):
    return _sph_z[J-1](n=n,z=z)
    

In [86]:
# All arguments are single numbers (for now)
#
def A(m,n,μ,ν,kdlj,θlj,φlj,r_ge_d,J):
    presum = exp(lgamma(2*n+1)-lgamma(n+2)+lgamma(2*ν+3)-lgamma(ν+2) 
                +lgamma(n+ν+m-μ+1)-lgamma(n-m+1)-lgamma(ν+μ+1))
    presum = presum * np.exp(1j*(μ-m)*φlj) * (-1)**m * 1j**(ν+n) / (4*n)
    qmax = q_max(-m,n,μ,ν)
    q = np.arange(qmax+1, dtype=int)
    # N.B. -m !!!!!!
    a1q = a_q(-m,n,μ,ν) # there is redundant calc. of qmax
    ã1q = a1q / a1q[0]
    p = n+ν-2*q
    print(qmax,p.shape,q.shape,a1q.shape)
    print("q",q)
    print("q",p)
    print("a1q",a1q)
    print("ã1q",ã1q)
    print("presum",presum)
    if(r_ge_d):
        J = 1
    zp = sph_zn(n+ν,kdlj,J)[p]
    print("zp",zp)
    Pp = lpmv(μ-m,p,math.cos(θlj))
    print("Pp",Pp)
    summandq = (n*(n+1) + ν*(ν+1) - p*(p+1)) * (-1)**q * ã1q * zp * Pp
    print("summandq",summandq)
    return presum * np.sum(summandq)
    

In [20]:
qmax, a0, aq_tilde = gaunt_hartmann(55,72,-15,-58)

In [27]:
a0*np.array(aq_tilde)

array([  2.64656585e+17,  -9.02060942e+17,   1.26120695e+18,
        -9.08154188e+17,   3.32877338e+17,  -3.99902372e+16,
        -9.37042460e+15,   1.93795427e+15,   4.15147444e+14,
        -4.50334569e+13,  -1.87310470e+13,  -6.89282471e+11,
         4.70652220e+11,   8.36238653e+10,   3.33171146e+08,
        -1.79374160e+09,  -2.62337431e+08,  -5.94969041e+06,
         3.03619342e+06,   4.52389705e+05,   2.13275693e+04,
        -1.46829323e+03,  -2.69213511e+02,  -1.39717873e+01,
         9.60086347e-03,   2.93425224e-02,   8.10789281e-04,
        -5.91092804e-06])

In [88]:
A(-15,55,-58,72,2.0,0.5,0.5,True,1)

OverflowError: math range error

In [31]:
sph_jn(4,6)

(array([-0.04656925, -0.16778992, -0.03732571,  0.13668516,  0.19679174]),
 array([ 0.16778992,  0.00936072, -0.14912707, -0.12844915, -0.02730795]))

In [36]:
sph_z(4,6,4)

array([-0.04656925+0.16002838j, -0.16778992-0.01989785j,
       -0.03732571-0.16997731j,  0.13668516-0.1217499j ,
        0.19679174+0.02793575j])

2.652528598121911e+32

In [38]:
factorial(30)

array(2.652528598121911e+32)

In [61]:
lpmv(2,np.array([4,5,6]),math.cos(0.5))

array([  7.56959726,  13.87749947,  20.24449652])

In [56]:
lpmn(2,5,math.cos(0.5))

(array([[  1.        ,   0.87758256,   0.65522673,   0.37330421,
           0.08188917,  -0.16928726],
        [  0.        ,  -0.47942554,  -1.26220648,  -2.05008768,
          -2.51500749,  -2.40342551],
        [  0.        ,   0.        ,   0.68954654,   3.0256701 ,
           7.56959726,  13.87749947]]),
 array([[  0.        ,   1.        ,   2.63274769,   4.27613365,
           5.24587717,   5.01313617],
        [  0.        ,   1.83048772,   3.3809357 ,   1.51637775,
          -6.18637661, -19.76961556],
        [  0.        ,   0.        ,  -5.26549537, -19.65680188,
         -36.62304916, -34.39685236]]))